In [53]:
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import time
import branca.colormap as cm

In [55]:
norske_kommuner = requests.get("https://api.kartverket.no/kommuneinfo/v1/kommuner/illustrasjonskart").json()
gdf = gpd.GeoDataFrame.from_features(norske_kommuner['features'], crs="EPSG:4326")

gdf.head()

,geometry,kommunenavn,kommunenummer
0,"POLYGON ((13.642 65.738, 13.754 65.776, 13.871...",Aarborte,1826
1,"POLYGON ((12.071 65.867, 12.156 65.888, 12.228...",Alstahaug,1820
2,"POLYGON ((22.217 70.039, 22.278 70.092, 22.604...",Alta,5601
3,"POLYGON ((10.156 61.948, 10.166 62.042, 10.392...",Alvdal,3428
4,"POLYGON ((4.777 60.734, 4.791 60.764, 5.039 60...",Alver,4631


In [59]:
kommuneliste = ",".join(gdf["kommunenummer"])

r = requests.get(f"https://frost.met.no/sources/v0.jsonld", auth=("c3801661-42b7-4986-a58c-a25e31779814",""))
stasjoner_json = r.json()

df_stasjoner = pd.DataFrame(stasjoner_json["data"])
df_stasjoner = df_stasjoner.dropna(subset=['municipalityId'])
df_stasjoner.head()




,@type,id,name,shortName,country,countryCode,geometry,masl,validFrom,county,countyId,municipality,municipalityId,ontologyId,stationHolders,externalIds,wigosId,shipCodes,wmoId,icaoCodes
0,SensorSystem,SN47230,ÅKRA UNGDOMSSKOLE,Åkra,Norge,NO,"{'@type': 'Point', 'coordinates': [5.1963, 59....",18.0,2013-10-29T00:00:00.000Z,ROGALAND,11.0,KARMØY,1149.0,0.0,[KARMØY KOMMUNE],[506131077],0-578-0-47230,NaN,NaN,NaN
1,SensorSystem,SN20952,STATFJORD C,Statfjord C,Norge,NO,"{'@type': 'Point', 'coordinates': [1.9025, 61....",NaN,1985-06-26T00:00:00.000Z,NORDSJØEN,26.0,NORDSJØEN,2600.0,0.0,[EQUINOR ASA],[LF6N],0-578-0-20952,[LF6N],NaN,NaN
2,SensorSystem,SN23670,E16 RYFOSS,E16 Ryfoss,Norge,NO,"{'@type': 'Point', 'coordinates': [8.8175, 61....",406.0,2018-01-23T00:00:00.000Z,INNLANDET,34.0,VANG,3454.0,0.0,[STATENS VEGVESEN],"[1755, 3000021]",0-578-0-23670,NaN,NaN,NaN
3,SensorSystem,SN59450,STADLANDET,Stadlandet,Norge,NO,"{'@type': 'Point', 'coordinates': [5.2115, 62....",75.0,1923-01-01T00:00:00.000Z,VESTLAND,46.0,STAD,4649.0,0.0,[MET.NO],"[10.249.1.80, 10.249.2.24]",0-578-0-59450,NaN,NaN,NaN
4,SensorSystem,SN55000,LUSTER - ORNES,Luster − Ornes,Norge,NO,"{'@type': 'Point', 'coordinates': [7.313199, 6...",4.0,1987-05-01T00:00:00.000Z,VESTLAND,46.0,LUSTER,4644.0,0.0,[NIBIO],NaN,0-578-0-55000,NaN,NaN,NaN


In [61]:
def stasjoner_i_kommune(kommune):
    kommunenummer = int(kommune)
    df_kommune_stasjoner = df_stasjoner[df_stasjoner["municipalityId"] == kommunenummer]
    stasjonsliste = ",".join(df_kommune_stasjoner["id"])
    if stasjonsliste == "":
        return np.nan
    return stasjonsliste


In [63]:
gdf["stasjonsliste"] = gdf["kommunenummer"].apply(stasjoner_i_kommune)
gdf.head()

,geometry,kommunenavn,kommunenummer,stasjonsliste
0,"POLYGON ((13.642 65.738, 13.754 65.776, 13.871...",Aarborte,1826,"SN77850,SN78800"
1,"POLYGON ((12.071 65.867, 12.156 65.888, 12.228...",Alstahaug,1820,"SN76780,SN76750,SN76530,SN76790"
2,"POLYGON ((22.217 70.039, 22.278 70.092, 22.604...",Alta,5601,"SN93070,SN94230,SN94020,SN94050,SN93200,SN9289..."
3,"POLYGON ((10.156 61.948, 10.166 62.042, 10.392...",Alvdal,3428,"SN8840,SN8880,SN9400"
4,"POLYGON ((4.777 60.734, 4.791 60.764, 5.039 60...",Alver,4631,"SN52505,SN52415,SN52400,SN52555,SN52390,SN52360"


In [65]:
def hent_temperatur(stasjonsliste):
    if stasjonsliste != "NaN":
        request_info_dict = {'sources': stasjonsliste, 'elements': 'air_temperature', 'referencetime': 'latest'}
        q = requests.get("https://frost.met.no/observations/v0.jsonld", request_info_dict, auth=("c3801661-42b7-4986-a58c-a25e31779814",""))
        if q.status_code == 200:
            temperaturdata_json = q.json()
            #print(temperaturdata_json)
            temperaturdata = [x['observations'][0]['value'] for x in temperaturdata_json['data'] if len(x['observations']) > 0 ]
            snitt_temperatur = np.mean(temperaturdata)
            #print(snitt_temperatur)
            return snitt_temperatur
    return np.nan

In [67]:
t1 = time.time()
gdf["temp"] = gdf["stasjonsliste"].apply(hent_temperatur)
t2 = time.time()
print(f"Hentet alle måledata på {t2-t1} sekunder")

Hentet alle måledata på 277.7088055610657 sekunder


In [77]:
gdf.head()

,geometry,kommunenavn,kommunenummer,stasjonsliste,temp
0,"POLYGON ((13.642 65.738, 13.754 65.776, 13.871...",Aarborte,1826,"SN77850,SN78800",13.40
1,"POLYGON ((12.071 65.867, 12.156 65.888, 12.228...",Alstahaug,1820,"SN76780,SN76750,SN76530,SN76790",15.40
2,"POLYGON ((22.217 70.039, 22.278 70.092, 22.604...",Alta,5601,"SN93070,SN94230,SN94020,SN94050,SN93200,SN9289...",8.85
3,"POLYGON ((10.156 61.948, 10.166 62.042, 10.392...",Alvdal,3428,"SN8840,SN8880,SN9400",10.70
4,"POLYGON ((4.777 60.734, 4.791 60.764, 5.039 60...",Alver,4631,"SN52505,SN52415,SN52400,SN52555,SN52390,SN52360",11.90


In [79]:
kart = folium.Map([65, 10], zoom_start=5)

tooltip = folium.GeoJsonTooltip(
    fields=["kommunenavn", "temp"],
    aliases=["Kommune:", "Temperatur:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

fargeskala = cm.LinearColormap(["blue", "white", "red"], vmin=-30, vmax=30)
fargeskala.caption = "Temperatur i celsius"

folium.GeoJson(gdf,
               style_function=lambda x: {
                   "fillColor": fargeskala(x["properties"]["temp"]) if x["properties"]["temp"] is not None else "black",
                   "fillOpacity": 0.8,
                   "weight": 1,
                   "color": "black",
               },
               tooltip=tooltip,
              ).add_to(kart)
kart.add_child(linear)
kart